# Import Library

In [1]:
!pip install ultralytics opencv-python tqdm huggingface_hub

  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.2.6 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.2.6 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.2.6 which is incompatible.


In [2]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 
# cuda 12.1 기준

Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
! pip install numpy==1.26.4 --force-reinstall
# ! pip install numpy==1.25.2 --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [4]:
import os
import json
import cv2
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import numpy as np

# 2. Json -> YOLO txt 변환

In [5]:
def convert_to_yolo(json_file_path, output_dir):
    """
    JSON 파일을 읽어 YOLO 포맷의 txt 파일로 변환합니다.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 이미지 크기 정보 추출
        resolution = data.get("meta", {}).get("Resolution", "0x0").split('x')
        if len(resolution) != 2:
            # print(f"Skipping {json_file_path}: Invalid resolution format.")
            return
            
        img_width, img_height = int(resolution[0]), int(resolution[1])
        if img_width == 0 or img_height == 0:
            # print(f"Skipping {json_file_path}: Image width or height is zero.")
            return

        # 어노테이션 정보 추출
        annotations = data.get("annotations", {}).get("Bbox Annotation", {})
        boxes = annotations.get("Box", [])
        
        if not boxes:
            # Bounding Box가 없는 경우, 빈 txt 파일 생성
            base_filename = os.path.basename(json_file_path)
            txt_filename = os.path.splitext(base_filename)[0] + ".txt"
            open(os.path.join(output_dir, txt_filename), 'w').close()
            return

        yolo_data = []
        for box in boxes:
            # YOLO 포맷으로 변환
            # 현재는 단일 클래스(0)로 처리
            class_id = 0  
            x, y, w, h = box['x'], box['y'], box['w'], box['h']
            
            x_center = (x + w / 2) / img_width
            y_center = (y + h / 2) / img_height
            width_norm = w / img_width
            height_norm = h / img_height
            
            yolo_data.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

        # YOLO 포맷으로 파일 저장
        base_filename = os.path.basename(json_file_path)
        txt_filename = os.path.splitext(base_filename)[0] + ".txt"
        
        with open(os.path.join(output_dir, txt_filename), 'w') as f:
            f.write("\n".join(yolo_data))

    except Exception as e:
        print(f"Error processing {json_file_path}: {e}")

def process_dataset(dataset_path):
    """
    'train'과 'val' 폴더에 대해 변환 작업을 수행합니다.
    """
    for split in ["train", "val"]:
        json_dir = os.path.join(dataset_path, split, "annotations")
        output_dir = os.path.join(dataset_path, split, "labels")

        if not os.path.exists(json_dir):
            print(f"Directory not found: {json_dir}")
            continue

        os.makedirs(output_dir, exist_ok=True)
        
        json_files = glob(os.path.join(json_dir, "*.json"))
        print(f"Found {len(json_files)} json files in {json_dir}")

        for json_file in tqdm(json_files, desc=f"Converting {split} JSONs to YOLO format"):
            convert_to_yolo(json_file, output_dir)
        
        print(f"Finished processing for {split} set. Labels are in {output_dir}")

In [9]:
# 한번만 실행
root_path = "C:/Users/User/Project_Big"
dataset_base_path = os.path.join(root_path, "custom_dataset")
process_dataset(dataset_base_path)

Found 160000 json files in C:/Users/User/Project_Big\custom_dataset\train\annotations


Converting train JSONs to YOLO format:   0%|          | 168/160000 [00:03<58:04, 45.86it/s]  


KeyboardInterrupt: 

# 3. YOLO Fine tuning

In [6]:
model = YOLO('yolov8n.pt')  # nano, 또는 yolov8s.pt 등 다른 가중치 사용 가능

In [7]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

PyTorch version: 2.5.1+cu121
CUDA available: False
CUDA version: 12.1
GPU name: No GPU


In [8]:
results = model.train(
    data=os.path.join('./data.yaml'),
    epochs=1,       # 필요에 따라 조정
    imgsz=640,
    batch=16,        # 메모리에 따라 조정
    device='cpu'         # GPU 사용
)


Ultralytics 8.3.168  Python-3.12.4 torch-2.5.1+cu121 CPU (13th Gen Intel Core(TM) i5-1340P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

100%|██████████| 755k/755k [00:00<00:00, 3.35MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

train: Scanning C:\Users\User\Project_Big\custom_dataset\train\labels... 160000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160000/160000 [15:51<00:00, 168.22it/s]


train: New cache created: C:\Users\User\Project_Big\custom_dataset\train\labels.cache
val: Fast image access  (ping: 0.20.0 ms, read: 1.30.4 MB/s, size: 36.8 KB)


val: Scanning C:\Users\User\Project_Big\custom_dataset\val\labels... 20000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [02:16<00:00, 146.46it/s]


val: New cache created: C:\Users\User\Project_Big\custom_dataset\val\labels.cache
Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      4.156       8.27      2.448         47        640:   3%|▎         | 329/10000 [45:29<22:17:00,  8.29s/it]


KeyboardInterrupt: 

In [20]:
import numpy as np
print(np.__version__)

2.2.6


# 4. 모델 추론

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
# 단일 이미지 추론
val_img_path = os.path.join(dataset_base_path, 'val/images/33_20210704_7454-0-0600.jpg')
results = model.predict(val_img_path, save=True, conf=0.25, device=0)

In [ ]:
# 전체 평가
metrics = model.val(
    data=os.path.join(root_path, 'data.yaml'),
    batch=16,
    imgsz=640,
    device=0
)
print(metrics)
# map50, map50-95, precision, recall 등 주요 지표 출력
